## Instalando dependencias necesarias

In [ ]:
%%capture
%pip install -U sentence-transformers pandas


## Entendiendo el dataset

In [ ]:
# https://www.kaggle.com/datasets/omarhanyy/imdb-top-1000?resource=download
# TODO: Cargar el archivo del dataset con  pandas
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv("IMDB top 1000.csv")



In [ ]:
# TODO: mostrar los primeros 5 registros de dataframe
print(df.head())

   Unnamed: 0                                              Title Certificate  \
0           0                 1. The Shawshank Redemption (1994)           R   
1           1                            2. The Godfather (1972)           R   
2           2                          3. The Dark Knight (2008)       PG-13   
3           3                   4. The Godfather: Part II (1974)           R   
4           4  5. The Lord of the Rings: The Return of the Ki...       PG-13   

  Duration                     Genre  Rate  Metascore  \
0  142 min                     Drama   9.3       80.0   
1  175 min              Crime, Drama   9.2      100.0   
2  152 min      Action, Crime, Drama   9.0       84.0   
3  202 min              Crime, Drama   9.0       90.0   
4  201 min  Action, Adventure, Drama   8.9       94.0   

                                         Description  \
0  Two imprisoned men bond over a number of years...   
1  The aging patriarch of an organized crime dyna...   
2  When 

## Usando Sentence Transformer para crear embeddings

In [ ]:
from sentence_transformers import SentenceTransformer  # Import the class

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(df.columns)

Index(['Unnamed: 0', 'Title', 'Certificate', 'Duration', 'Genre', 'Rate',
       'Metascore', 'Description', 'Cast', 'Info'],
      dtype='object')


In [ ]:
embeddings = model.encode(df['Title'],batch_size=64,show_progress_bar=True)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
df['embeddings'] = embeddings.tolist()

## Calculando la similitud usando la métrica de similitud por coseno

In [ ]:
def compute_similarity(example, query_embedding):
    embedding = example['embeddings']
    similarity = util.cos_sim(embedding, query_embedding).item()
    return similarity


## Ejecuntando la búsqueda

In [ ]:
from sentence_transformers import util

def search_by_multiple_fields(df, query, fields=['Title', 'Genre']):
  """
  Busca en un DataFrame por múltiples campos.

  Args:
    df: El DataFrame de películas.
    query: La consulta de búsqueda.
    fields: Una lista de los nombres de los campos a buscar.

  Returns:
    Un DataFrame ordenado por similitud.
  """

  query_embedding = model.encode([query])[0]

  df['combined_text'] = df[fields].apply(lambda x: ' '.join(x.dropna()), axis=1)
  # Apply the compute_similarity function to each row of the DataFrame
  df['similarity'] = df.apply(lambda row: compute_similarity(row, query_embedding), axis=1)
  return df.sort_values(by='similarity', ascending=False)

# Ejemplo de uso:
results = search_by_multiple_fields(df, 'time travel sci-fi', fields=['Title', 'Genre'])
print(results.head()['Title'])

46         47. Back to the Future (1985)
20               21. Interstellar (2014)
114    115. 2001: A Space Odyssey (1968)
44            45. Cinema Paradiso (1988)
15                 16. The Matrix (1999)
Name: Title, dtype: object


In [ ]:
from sentence_transformers import util
query_embedding = model.encode(['a travel time adventure'])[0]
df['similarity'] = df.apply(lambda x: compute_similarity(x, query_embedding), axis=1)
df = df.sort_values(by='similarity', ascending=False)

df.head()['Title']

,Title
798,301. Nights of Cabiria (1957)
498,301. Nights of Cabiria (1957)
298,301. Nights of Cabiria (1957)
898,301. Nights of Cabiria (1957)
698,301. Nights of Cabiria (1957)
